In [1]:
import sys, os
import ROOT
import numpy as np
import xgboost as xgb
import pandas as pd
from CMS_lumi import CMS_lumi
from colorPrint import *

ROOT.gROOT.LoadMacro("../interface/tdrstyle.C")
ROOT.gROOT.ProcessLine("setTDRStyle();")

Welcome to JupyROOT 6.24/02


/home/chenghan/.conda/envs/hdalitz2/lib/python3.8/site-packages/ROOT/_facade.py:150: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  return _orig_ihook(name, *args, **kwds)
/home/chenghan/.conda/envs/hdalitz2/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


0

In [ ]:
iBE = 0 # 0 for EB and 1 for EE

features = [
    "rho",
    "nVtx",
    "eleSCEta_Lead",
    "eleSCPhi_Lead",
    "eleSCRawEn_Lead",
    "eleCalibPt_Lead",

    "eledEtaAtVtx_Lead",
    "eledPhiAtVtx_Lead",
    "elePtError_Lead",
    "eleHoverE_Lead",
    "eleEoverP_Lead",
    "eleEoverPout_Lead",
    "eleEoverPInv_Lead",

    "eleSCEtaWidth_Lead",
    "eleSCPhiWidth_Lead",
    "eleSigmaIEtaIEtaFull5x5_Lead",
    "eleSigmaIPhiIPhiFull5x5_Lead",
    "eleR9Full5x5_Lead",
    "eleBrem_Lead",

    "gsfPtSum_Lead",
    "gsfPtRatio_Lead",
    "diTrkPt",
    "gsfDeltaR_Lead"
]
if iBE == 1:
    features.append("eleESEnToRawE_Lead")


cut_base = "elePresel_Lead == 1 && eleCalibPt_Lead > 25 && category == 2"
cut_region = "fabs(eleSCEta_Lead) < 1.479"
if iBE == 1:
    cut_region = "fabs(eleSCEta_Lead) >= 1.479 && fabs(eleSCEta_Lead) < 2.5"
# cut_target = "target < 2 && target > 0"

ROOT.EnableImplicitMT(20)
data = ROOT.RDataFrame("miniTree", "../reg_signal.root")\
           .Define("target",    
                #    "diGenEle.Pt()/eleCalibPt_Lead"
                   "diGenEle.Pt()/elePt_Lead"
            )\
           .Define("diTrkPt",   "diTrk.Pt()")\
           .Define("RatioRL",   "(eleEright_Lead - eleEleft_Lead)/(eleEright_Lead + eleEleft_Lead)")\
           .Define("RatioBT",   "(eleEtop_Lead - eleEbottom_Lead)/(eleEtop_Lead + eleEbottom_Lead)")\
           .Define("eleCalibPtToRawE", "eleCalibPt_Lead/eleSCRawEn_Lead")\
           .Define("diTrkPtToRawE", "diTrkPt/eleSCRawEn_Lead")\
           .Define("eleEmaxToRawE", "eleEmax_Lead/eleSCRawEn_Lead")\
           .Define("eleE2ndToRawE", "eleE2nd_Lead/eleSCRawEn_Lead")\
           .Define("eleE2x2ToRawE", "eleE2x2Full5x5_Lead/eleSCRawEn_Lead")\
           .Define("eleE2x5ToRawE", "eleE2x5Full5x5_Lead/eleSCRawEn_Lead")\
           .Define("eleE1x3ToRawE", "eleE1x3Full5x5_Lead/eleSCRawEn_Lead")\
           .Define("genPt",     "diGenEle.Pt()")\
           .Filter("{} && {}".format(cut_base, cut_region))\